In [ ]:
import pandas  as pd
import numpy as np
import matplotlib.pyplot  as plt
from sklearn.utils import shuffle
import cv2

import tensorflow as tf 
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D

import os, json
from PIL import Image
import seaborn as sns
import datetime
import albumentations as A

In [ ]:
data_path = "../input/cassava-leaf-disease-classification/"
train_csv_data_path = data_path+"train.csv"
label_json_data_path = data_path+"label_num_to_disease_map.json"
images_dir_data_path = data_path+"train_images"

In [ ]:
import os
print('Train images: %d' %len(os.listdir(
    os.path.join(data_path, "train_images"))))

In [ ]:
with open(os.path.join(data_path, "label_num_to_disease_map.json")) as file:
    print(json.dumps(json.loads(file.read()), indent=4))

In [ ]:
train_csv = pd.read_csv(train_csv_data_path)
train_csv['label'] = train_csv['label'].astype('string')

label_class = pd.read_json(label_json_data_path, orient='index')
label_class = label_class.values.flatten().tolist()

In [ ]:
sns.set_style("whitegrid")
fig, ax = plt.subplots(figsize = (6, 4))

for i in ['top', 'right', 'left']:
    ax.spines[i].set_visible(False)
ax.spines['bottom'].set_color('black')

sns.countplot(train_csv.label, edgecolor = 'black',
              palette = reversed(sns.color_palette("viridis", 5)))
plt.xlabel('Classes', fontfamily = 'serif', size = 15)
plt.ylabel('Count', fontfamily = 'serif', size = 15)
plt.xticks(fontfamily = 'serif', size = 12)
plt.yticks(fontfamily = 'serif', size = 12)
ax.grid(axis = 'y', linestyle = '--', alpha = 0.9)
plt.show()

In [ ]:
def visualize(image):
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(image)
    
def visualize_multiple(nrows, ncols, img, transform):
    fig, axes = plt.subplots(nrows,ncols)
    fig.set_figheight(15)
    fig.set_figwidth(15)
    num_iter = 0
    for row in range(nrows):
        for col in range(ncols):
            augmented_img = transform[num_iter](image=img)['image']
            axes[row,col].imshow(augmented_img)
            axes[row,col].grid(False)
            axes[row,col].set_xticks([])
            axes[row,col].set_yticks([])
            num_iter += 1
    return fig, axes

In [ ]:
#seed_everything(100)
img = cv2.imread('../input/cassava-leaf-disease-classification/train_images/100042118.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
visualize(img)

# # Blur

In [ ]:
blur_limits = np.arange(3,39,4)
transform = [A.Blur(p=1, blur_limit=[limit,limit], always_apply=True) for limit in blur_limits]
fig, axes = visualize_multiple(3,3,img,transform)

num_iter = 0
for row in range(3):
    for col in range(3):
        text = 'Blur kernel size: ({}, {})'.format(blur_limits[num_iter], blur_limits[num_iter])
        axes[row, col].set_title(text)
        num_iter += 1

# # CLAHE

In [ ]:
params = np.arange(3,30,3)
transform = [A.CLAHE(clip_limit=[param, param], tile_grid_size=(param, param), always_apply=True) for param in params]
fig, axes = visualize_multiple(3,3,img,transform)

num_iter = 0
for row in range(3):
    for col in range(3):
        text = 'Clip limit: ({}, {}), tile grid size: ({}, {})'.format(params[num_iter], params[num_iter], params[num_iter], params[num_iter])
        axes[row, col].set_title(text)
        num_iter += 1

In [ ]:
print("Label names :")
for i, label in enumerate(label_class):
    print(f" {i}. {label}")

In [ ]:
train_csv.head()

In [ ]:
BATCH_SIZE = 18
IMG_SIZE = 224

In [ ]:
from keras.models import load_model
# model_model = load_model('../input/resnet/ResNet50 (1).h5') # for resnet50 224px
# model_model = load_model('../input/private01/EfficientNetB7.h5') # for EfficientNetB7 300px
# model_model = load_model('../input/private02/ResNet152(1).h5') # for ResNet152 224px
model_model = load_model('../input/4feb2/4Feb-2.h5') # for ResNet152 224px

In [ ]:
model_model.summary()

In [ ]:
test_img_path = data_path+"test_images/2216849948.jpg"

img = cv2.imread(test_img_path)
resized_img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)).reshape(-1, IMG_SIZE, IMG_SIZE, 3)/255

plt.figure(figsize=(8,4))
plt.title("TEST IMAGE")
plt.imshow(resized_img[0])

In [ ]:
preds = []
ss = pd.read_csv(data_path+'sample_submission.csv')

for image in ss.image_id:
    img = tf.keras.preprocessing.image.load_img(data_path+'test_images/' + image)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.preprocessing.image.smart_resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.reshape(img, (-1, IMG_SIZE, IMG_SIZE, 3))
    prediction = model_model.predict(img/255)
    preds.append(np.argmax(prediction))

my_submission = pd.DataFrame({'image_id': ss.image_id, 'label': preds})
my_submission.to_csv('submission.csv', index=False) 

In [ ]:
# Submission file ouput
print("Submission File: \n---------------\n")
print(my_submission.head())